In [4]:
#load pdf document
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("unit2.pdf")
docs= loader.load()


In [5]:
#chunk the given document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap= 200)
documents= text_splitter.split_documents(docs)


In [6]:
#vector embedding and vector store
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
db= FAISS.from_documents(documents[:30], OllamaEmbeddings(model= "llama2"))

In [7]:
#initialize llm model
from langchain_ollama import OllamaLLM
llm= OllamaLLM(model= "llama2")

In [8]:
#design chatPrompt template
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template(
    """
Answer the given question based on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question: {input}
"""
)

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain= create_stuff_documents_chain(llm, prompt)

In [10]:
#adding retrival as a interface to our vectore store db
retriever= db.as_retriever()


In [11]:
#create retriever chain that combines retriever and document chain
from langchain.chains import create_retrieval_chain
retrival_chain= create_retrieval_chain(retriever, document_chain )

In [12]:
response= retrival_chain.invoke({"input": "What is an intelligent agent?"})

In [13]:
print(response)

{'input': 'What is an intelligent agent?', 'context': [Document(id='510723d0-dc56-4c77-bb8a-835d225030d4', metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2025-06-07T10:23:45+05:45', 'title': '', 'author': 'HP', 'moddate': '2025-06-07T10:23:45+05:45', 'source': 'unit2.pdf', 'total_pages': 45, 'page': 8, 'page_label': '9'}, page_content='inputs it receives from its environment.\nAgent = Architecture + Agent Program\n• The overall structure of an AI agent can be understood as a combination of both the architecture and \nthe agent program. The architecture provides the physical infrastructure, while the agent program \ndictates the decision-making and actions of the agent based on its perceptual inputs.'), Document(id='93f92da7-55db-4d55-bcd6-30d1541b9185', metadata={'producer': 'Microsoft® PowerPoint® 2016', 'creator': 'Microsoft® PowerPoint® 2016', 'creationdate': '2025-06-07T10:23:45+05:45', 'title': '', 'author': 'HP', 'm

In [14]:
print(response['answer'])

An intelligent agent is a computational entity that perceives its environment through sensors, acts upon the environment through actuators, and learns from experiences. In other words, it is a program that can act in a dynamic environment to achieve specific goals. The key features of an intelligent agent are:

1. Perception: Agents receive inputs from their environment through sensors, which can be visual, auditory, tactile, or any other type of sensor. These inputs provide the basis for decision-making and action.
2. Action: Agents use actuators to affect the environment. Actuators can be mechanical, electrical, or software-based and can take various forms such as a screen display, keyboard input, or a robotic arm.
3. Learning: Intelligent agents learn from their experiences through machine learning algorithms, statistical models, or other techniques. This enables them to adapt to changing environments and improve their performance over time.
4. Decision-making: Agents use reasoning,